In [ ]:
!cp ../input/talibinstall/ta-lib-0.4.0-src.tar.gzh  ./ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz > null
!cd ta-lib && ./configure --prefix=/usr > null && make  > null && make install > null
!cp ../input/talibinstall/TA-Lib-0.4.21.tar.gzh TA-Lib-0.4.21.tar.gz
!pip install TA-Lib-0.4.21.tar.gz > null
!pip install ../input/talibinstall/numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl >null

!rm -rf ./ta-lib
!rm  ./TA-Lib-0.4.21.tar.gz
!rm  ./ta-lib-0.4.0-src.tar.gz
!rm  ./null

import talib as ta

In [ ]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import numpy as np
from scipy.special import comb
from itertools import combinations

class CombinatorialPurgedGroupKFold():
    def __init__(self, n_splits = 6, n_test_splits = 2, purge = 1, pctEmbargo = 0.01, **kwargs):
        self.n_splits = n_splits
        self.n_test_splits = n_test_splits
        self.purge = purge
        self.pctEmbargo = pctEmbargo
        
    def split(self, X, y = None, groups = None):
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
            
        u, ind = np.unique(groups, return_index = True)
        unique_groups = u[np.argsort(ind)]
        n_groups = len(unique_groups)
        group_dict = {}
        for idx in range(len(X)):
            if groups[idx] in group_dict:
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
                
        n_folds = comb(self.n_splits, self.n_test_splits, exact = True)
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
            
        mbrg = int(n_groups * self.pctEmbargo)
        if mbrg < 0:
            raise ValueError(
                "The number of 'embargoed' groups should not be negative")
        
        split_dict = {}
        group_test_size = n_groups // self.n_splits
        for split in range(self.n_splits):
            if split == self.n_splits - 1:
                split_dict[split] = unique_groups[int(split * group_test_size):].tolist()
            else:
                split_dict[split] = unique_groups[int(split * group_test_size):int((split + 1) * group_test_size)].tolist()
        
        for test_splits in combinations(range(self.n_splits), self.n_test_splits):
            test_groups = []
            banned_groups = []
            for split in test_splits:
                test_groups += split_dict[split]
                banned_groups += unique_groups[split_dict[split][0] - self.purge:split_dict[split][0]].tolist()
                banned_groups += unique_groups[split_dict[split][-1] + 1:split_dict[split][-1] + self.purge + mbrg + 1].tolist()
            train_groups = [i for i in unique_groups if (i not in banned_groups) and (i not in test_groups)]

            train_idx = []
            test_idx = []
            for train_group in train_groups:
                train_idx += group_dict[train_group]
            for test_group in test_groups:
                test_idx += group_dict[test_group]
            yield train_idx, test_idx

In [ ]:
import gc, os
import numpy as np
import pandas as pd

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

import plotly as py
import plotly.graph_objs as go
import plotly.tools as tools

from tqdm.notebook import tqdm

In [ ]:
import numpy.polynomial.hermite as Herm
import math
from tensorflow.python.ops import math_ops
from scipy import stats
import tensorflow_probability as tfp

In [ ]:
path_s = "../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/"
path_t = "../input/jpx-tokyo-stock-exchange-prediction/train_files/"
path_e = "../input/jpx-tokyo-stock-exchange-prediction/example_test_files/"

In [ ]:
prices1 = pd.read_csv(f"{path_s}stock_prices.csv") #2021
prices2 = pd.read_csv(f"{path_t}stock_prices.csv") #2017
prices3 = pd.read_csv(f"{path_t}secondary_stock_prices.csv")

prices1.shape , prices2.shape, prices3.shape

In [ ]:
prices2['Target'].isnull().sum(), prices1['Target'].isnull().sum(), prices3['Target'].isnull().sum()

In [ ]:
#train2 = prices2.dropna(subset=['Target'])
#train1 = prices1.dropna(subset=['Target'])

train2, train1 = prices2.fillna(0), prices1.fillna(0)

In [ ]:
train2['Target'].isnull().sum(), train1['Target'].isnull().sum()

In [ ]:
df = pd.concat([train2,train1])

In [ ]:
def prep_prices(prices):
    prices.fillna(0,inplace=True)
    return prices

In [ ]:
#simple units
hbar = 1.0
m    = 1.0
w    = 1.0

def hermite(x, n):
    xi             = np.sqrt(m*w/hbar)*x
    herm_coeffs    = np.zeros(n+1)
    herm_coeffs[n] = 1
    return Herm.hermval(xi, herm_coeffs)

def stationary_state(x,n):
    xi        = np.sqrt(m*w/hbar)*x
    prefactor = 1.0/math.sqrt(2.0**n * math.factorial(n)) * (m*w/(np.pi*hbar))**(0.25)
    psi       = prefactor * np.exp(- xi**2 / 2) * hermite(x,n)
    return psi

In [ ]:
NDAYS = 180
lastdays = df[df["Date"]>=df.Date.iat[-2000*NDAYS]].reset_index(drop=True)

In [ ]:
lastdays = pd.DataFrame(df.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]

In [ ]:
def daily_standardize(df,col):
    avg = df[[col,'Date']].groupby('Date').mean()
    avg.columns = ['avg']
    avg['Date'] = avg.index
    avg = avg.reset_index(drop=True)
    std = df[[col,'Date']].groupby('Date').std()
    std.columns = ['std']
    std['Date'] = std.index
    std = std.reset_index(drop=True)
    df = pd.merge(df, avg, on='Date')
    df = pd.merge(df,std,on='Date')
    df[col] = (df[col] - df['avg'])/df['std']
    df = df.drop(['avg','std'],axis=1)
    df[col] = df[col].fillna(0)
    return df

In [ ]:
from decimal import *
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    price.set_index("Date", inplace=True)
    return price

In [ ]:
def adjust_features(df):
    df=df.copy()
    col='AdjustedClose'
    periods=[3,5,8,12,15,26,30,35,40,45,50,60,]
    for period in periods:
        df.loc[:,"Return_{}Day".format(period)] = df.groupby("SecuritiesCode")[col].pct_change(period)
        df.loc[:,"MovingAvg_{}Day".format(period)] = df.groupby("SecuritiesCode")[col].rolling(window=period).mean().values
        df.loc[:,"ExpMovingAvg_{}Day".format(period)] = df.groupby("SecuritiesCode")[col].ewm(span=period,adjust=False).mean().values
        df.loc[:,"Volatility_{}Day".format(period)] = np.log(df[col]).groupby(df["SecuritiesCode"]).diff().rolling(period).std()
    return df

In [ ]:
def get_features(df, tr=True):    
    df = adjust_price(df)
    df = adjust_features(df)
    df = prep_prices(df)
    
    scale_features = df.columns.drop(['SecuritiesCode','Target'])    
    #df[scale_features] = MinMaxScaler(feature_range=(0,1)).fit_transform(df[scale_features]) 
    df[scale_features] = RobustScaler().fit_transform(df[scale_features]) 
    
    df['upper_Shadow']   = df['High'] - np.maximum(df['Close'], df['Open'])
    df['lower_Shadow']   = np.minimum(df['Close'], df['Open']) - df['Low'] 

    # The Golden Ratio Multiplier 
    df['GRM_0']    = (ta.MA(df['Close'], timeperiod=350, matype=0)) 
    df['GRM_1']    = (ta.MA(df['Close'], timeperiod=350, matype=0))*1.6  
    df['GRM_2']    = (ta.MA(df['Close'], timeperiod=350, matype=0))*2
    df['GRM_3']    = (ta.MA(df['Close'], timeperiod=350, matype=0))*3

    df['Pi_Cycle'] = ta.MA(df['Close'], timeperiod=111, matype=0) 
    
    # Momentum
    df['RSI_14'] = ta.RSI(df['Close'], timeperiod=14)
    df['RSI_24'] = ta.RSI(df['Close'], timeperiod=24)
    df['RSI_42'] = ta.RSI(df['Close'], timeperiod=42)
    
    df['RSI1']   = df['RSI_14'].shift(-1) 
    df['RSI4']   = df['RSI_14'].shift(-4) 
    df['RSI7']   = df['RSI_14'].shift(-7) 
    df['RSI10']  = df['RSI_14'].shift(-10) 
    df['RSI13']  = df['RSI_14'].shift(-13) 
    df['RSI16']  = df['RSI_14'].shift(-16) 
    
    df['MACD_12'], df['macdsignal_12'], df['MACD_HIST_12'] = ta.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9) 
    df['MACD_48'], df['macdsignal_48'], df['MACD_HIST_48'] = ta.MACD(df['Close'], fastperiod=48, slowperiod=104, signalperiod=36)
    
    df['macdsignal1'] = df['macdsignal_12'].shift(-1)
    df['macdsignal4'] = df['macdsignal_12'].shift(-4)
    df['macdsignal7'] = df['macdsignal_12'].shift(-7)
    df['MACD_HIST1']  = df['MACD_HIST_12'].shift(-1) 
    df['MACD_HIST4']  = df['MACD_HIST_12'].shift(-4) 
    df['MACD_HIST7']  = df['MACD_HIST_12'].shift(-7) 
    df['ROCP']     = ta.ROCP(df['Open'])
    df['momentam'] = ta.MOM(df['Open'])
    df['CMO']      = ta.CMO(df['Open']) 
    df['PPO']      = ta.PPO(df['Open'])
    df['SAR']       = ta.SAR(df['High'], df['Low'], acceleration=0, maximum=0) 
    df['DI_minus']  = ta.MINUS_DI(df['High'], df['Low'],np.array(df.loc[:, 'Close']), timeperiod=14) 
    df['DI_minus1'] = df['DI_minus'].shift(-1) 
    df['DI_minus4'] = df['DI_minus'].shift(-4) 
    df['DI_minus7'] = df['DI_minus'].shift(-7)  
    df['adx']    = ta.ADX(df['High'], df['Low'],np.array(df.loc[:, 'Close']),timeperiod=14) 
    df['adx1']   = df['adx'].shift(-1) 
    df['adx4']   = df['adx'].shift(-4) 
    df['adx+1']  = df['adx'].shift(1) 
    df['adx7']   = df['adx'].shift(-7)
    df['DI_plus']   = ta.PLUS_DI(df['High'], df['Low'],np.array(df.loc[:, 'Close']), timeperiod=14) 
    df['DI_plus1']  = df['DI_plus'].shift(-1) 
    df['DI_plus4']  = df['DI_plus'].shift(-4) 
    df['DI_plus7']  = df['DI_plus'].shift(-7) 
    df['DI_plus10'] = df['DI_plus'].shift(-10)
    df['APO']      = ta.APO(df['Open'])
    df['APO1']     = df['APO'].shift(-1)
    df['APO4']     = df['APO'].shift(-4)
    df['APO7']     = df['APO'].shift(-7)
    df['ROCR100']  = ta.AD(df['High'], df['Low'], df['Close'], df['Volume'])
    df['OBV']      = ta.OBV(df['Close'], df['Volume'])
    df['ADOSC']    = ta.ADOSC(df['High'], df['Low'], df['Close'], df['Volume'], fastperiod=3, slowperiod=10)
    df['TSF']      = ta.TSF(df['Close'], timeperiod=14) 
    df['TSF-14']   = ta.TSF(df['Close'], timeperiod=14).shift(-14)
    df['TSF-7']    = ta.TSF(df['Close'], timeperiod=14).shift(-7)
    df['ATR']    = ta.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['NATR']   = ta.NATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    df['s_avg']      = df['SecuritiesCode'].apply(get_avg)
    df['qhm_115v']   = stationary_state(df['s_avg'], 115) 
    
    ema_set = [3,5,8,12,15,26,30,35,40,45,50,60, 100,200]
    # EMA
    for i in range(len(ema_set)):        
        sma = df['Close'].rolling(ema_set[i]).mean()
        ema = sma.ewm(span=ema_set[i], adjust=False).mean()
        df["EMA_C_%d"%(ema_set[i])] = ema
                                                 
        df = prep_prices(df)
    
    if tr:
        df = df.drop(['RowId','AdjustmentFactor','ExpectedDividend','SupervisionFlag','AdjustedClose'],axis=1)
    return df

In [ ]:
train_df = get_features(df)
groups   = pd.factorize(train_df.index)[0]
train    = train_df.reset_index(drop=True)

In [ ]:
securities_code = train.pop("SecuritiesCode")
y               = train.pop("Target")

In [ ]:
corrs = list()
for col in tqdm(train.columns):
    corr = np.corrcoef(y, train[col])[0][1]
    corrs.append(corr)

In [ ]:
feat_importances = pd.Series(corrs, index=train.columns)
labels = feat_importances.index
values = feat_importances.value_counts().index
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.6)])
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig['layout'].update(height=800, width=800)
fig.show()

In [ ]:
select_n = 16
positive_feat = feat_importances.nlargest(select_n)
negative_feat = feat_importances.nlargest(len(feat_importances))[-select_n:]

In [ ]:
fig = go.Figure(data=[go.Pie(labels=positive_feat.index, values=positive_feat.value_counts().index, hole=.6)])
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig['layout'].update(height=800, width=800, title={
                                                     'text': "Positive Feature",
                                                     'y':0.95,
                                                     'x':0.42,
                                                     'xanchor': 'center',
                                                     'yanchor': 'top' })
fig.show()

fig = go.Figure(data=[go.Pie(labels=negative_feat.index, values=negative_feat.value_counts().index*-1, hole=.6)])
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig['layout'].update(height=800, width=800, title={
                                                     'text': "Negative Feature",
                                                     'y':0.95,
                                                     'x':0.42,
                                                     'xanchor': 'center',
                                                     'yanchor': 'top' })
fig.show()

In [ ]:
feats = pd.concat([positive_feat,negative_feat]).index
feats.shape[-1]

In [ ]:
train = train[feats]

In [ ]:
train

In [ ]:
'''
from sklearn import decomposition
pca    = decomposition.PCA(n_components=34)
df_pca = pca.fit_transform(train)
'''

In [ ]:
def create_record(i):
    dic = {}
    dic[f"features"] = tf.train.Feature(float_list=tf.train.FloatList(value=list(train.iloc[i])))
    dic["securities_code"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[securities_code.iloc[i]]))
    dic["Target"] = tf.train.Feature(float_list=tf.train.FloatList(value=[y.iloc[i]]))
    record_bytes = tf.train.Example(features=tf.train.Features(feature=dic)).SerializeToString()
    return record_bytes
    
def decode_function(record_bytes):
      return tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      {
          "features": tf.io.FixedLenFeature([feats.shape[-1]], dtype=tf.float32),
          "securities_code": tf.io.FixedLenFeature([], dtype=tf.int64),
          "Target": tf.io.FixedLenFeature([], dtype=tf.float32)
      }
  )

In [ ]:
%%time
import time
n_splits = 5
n_test_splits = 2
kfold = CombinatorialPurgedGroupKFold(n_splits, n_test_splits)
for fold, (train_indices, test_indices) in enumerate(kfold.split(train, groups=groups)):
    print("=" * 100)
    print(f"Fold {fold}")
    print("=" * 100)
    print("Train Sample size:", len(train_indices))
    print("Test Sample size:", len(test_indices))
    train_save_path = f"fold_{fold}_train.tfrecords"
    begin = time.time()
    print(f"Creating {train_save_path}")
    with tf.io.TFRecordWriter(train_save_path) as file_writer:
        for i in train_indices:
            file_writer.write(create_record(i))
    print("Elapsed time: %.2f"%(time.time() - begin))
    begin = time.time()
    print(f"Creating {train_save_path}")
    test_save_path = f"fold_{fold}_test.tfrecords"
    with tf.io.TFRecordWriter(test_save_path) as file_writer:
        for i in test_indices:
            file_writer.write(create_record(i))
    print("Elapsed time: %.2f"%(time.time() - begin))

In [ ]:
securities_codes = securities_code.unique()
securities_code_df = pd.DataFrame({"securities_code": securities_codes})
securities_code_df.to_csv("securities_codes.csv", index=False)